# Projections

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Categories

In [2]:
SKATER_CATEGORIES = ['goals', 'assists', 'plus_minus', 'powerplay_points', 'shots_on_goal', 'hits', 'blocks']
GOALIE_CATEGORIES = ["wins", "save_percentage", "goals_against_average", "saves", "shutouts"]

### Players

In [3]:
df = pd.read_csv("../data/adp-yahoo_com.csv")
df = df[["team", "name", "position"]]

df.head()

,team,name,position
0,Edm,Connor McDavid,C
1,Col,Nathan MacKinnon,C
2,Edm,Leon Draisaitl,"C,LW"
3,NYR,Artemi Panarin,LW
4,Was,Alex Ovechkin,LW


### Projection Data

In [4]:
prj = pd.concat([
    pd.read_csv("../data/projections-cbs_com.csv"), 
    pd.read_csv("../data/projections-dailyfaceoff_com.csv"), 
    pd.read_csv("../data/projections-yahoo_com.csv")
])

In [5]:
prj = pd.concat([
    pd.read_csv("../data/projections-cbs_com.csv"), 
    pd.read_csv("../data/projections-dailyfaceoff_com.csv"), 
    pd.read_csv("../data/projections-yahoo_com.csv")
])

# fix weird assists issue
prj["assists"] = prj["assists"].apply(pd.to_numeric, errors="coerce")

# goals against average is a negative thing
prj["goals_against_average"] = -prj["goals_against_average"]

# reduce
prj = (
    prj
    [["name"] + SKATER_CATEGORIES + GOALIE_CATEGORIES]
    .groupby("name")
    .mean()
    .round(3)
    .reset_index()
)

prj.head()

,name,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,blocks,wins,save_percentage,goals_against_average,saves,shutouts
0,Aaron Ekblad,7.487,20.350,5.727,6.207,126.403,46.40,69.9,NaN,NaN,NaN,NaN,NaN
1,Adam Boqvist,7.400,16.300,-4.100,2.800,83.400,50.00,65.2,NaN,NaN,NaN,NaN,NaN
2,Adam Fox,6.543,24.153,11.560,8.923,102.883,29.55,74.0,NaN,NaN,NaN,NaN,NaN
3,Adam Gaudette,10.460,16.725,-2.850,5.205,68.205,31.10,21.5,NaN,NaN,NaN,NaN,NaN
4,Adam Henrique,16.953,15.497,-0.120,6.877,109.103,53.55,39.1,NaN,NaN,NaN,NaN,NaN


### Export

In [6]:
df = pd.merge(df, prj, how="inner", on="name")
df.head()

,team,name,position,goals,assists,plus_minus,powerplay_points,shots_on_goal,hits,blocks,wins,save_percentage,goals_against_average,saves,shutouts
0,Edm,Connor McDavid,C,30.353,52.693,2.787,23.687,177.727,25.65,22.5,NaN,NaN,NaN,NaN,NaN
1,Col,Nathan MacKinnon,C,28.653,44.883,8.693,21.733,252.167,36.20,20.8,NaN,NaN,NaN,NaN,NaN
2,Edm,Leon Draisaitl,"C,LW",33.487,47.400,-1.777,22.997,164.293,29.80,16.4,NaN,NaN,NaN,NaN,NaN
3,NYR,Artemi Panarin,LW,24.137,45.087,16.927,14.747,159.880,15.20,13.4,NaN,NaN,NaN,NaN,NaN
4,Was,Alex Ovechkin,LW,33.417,20.877,-1.330,14.950,242.383,143.45,22.4,NaN,NaN,NaN,NaN,NaN


In [7]:
df.to_csv("../data/projections-generated.csv", index=False)